# L7: Build a Crew to Tailor Job Applications


## SETUP


In [37]:
from crewai import Agent, Task, Crew
import warnings

warnings.filterwarnings("ignore")

In [38]:
# setup openai, serper api
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

## SETUP FOR OLLAMA


https://docs.crewai.com/how-to/LLM-Connections/#ollama-integration


In [39]:
# from langchain_openai import ChatOpenAI
# import os

# OPENAI_API_BASE = "http://localhost:11434/v1"
# OPENAI_MODEL_NAME = "crewai-llama3"  # Adjust based on available model
# OPENAI_API_KEY = "NA"

# llm_ollama = ChatOpenAI(model=OPENAI_MODEL_NAME, base_url=OPENAI_API_BASE)

# llm_ollama.invoke("What is your name?")

## TOOLS


In [40]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool, MDXSearchTool, SerperDevTool

resume_md_path = "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Interviews/subproj_Resume/files_proj_Resume/Raghu/Resume/raghuvamsi_ayapilla_resume_2024.md"

tool_search = SerperDevTool()
tool_scrape = ScrapeWebsiteTool()
tool_read_resume = FileReadTool(file_path=resume_md_path)
tool_semantic_search_resume = MDXSearchTool(mdx=resume_md_path)

In [41]:
# # display the resume
# from IPython.display import Markdown, display

# display(Markdown(resume_md_path))

## AGENTS


In [42]:
# Researcher
agent_researcher = Agent(
    role="Tech Job Researcher",
    goal="""
Make sure to do amazing analysis on job posting to help job applicants
""",
    backstory=(
        """
As a Job Researcher, your prowess in navigating and extracting critical information from job postings is unmatched. Your skills help pinpoint the necessary qualifications and skills sought by employers, forming the foundation for effective application tailoring
"""
    ),
    tools=[tool_scrape, tool_search],
    verbose=True,
)

In [43]:
# Profiler
agent_profiler = Agent(
    role="Personal Profiler for {JOB_PROFILE}s",
    goal="""Do incredible research on job applicants to help them stand out in the job market""",
    backstory=(
        """
Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive personal and professional profiles, laying the groundwork for personalized resume enhancements.
"""
    ),
    tools=[tool_scrape, tool_search, tool_read_resume, tool_semantic_search_resume],
    verbose=True,
)

In [44]:
# Resume Strategist
agent_resume_strategist = Agent(
    role="Resume Strategist for {JOB_PROFILE}s",
    goal="""
Find all best ways to make an {JOB_PROFILE}'s resume more professional and stand out in the current job market.
""",
    backstory=(
        """
With a strategic mind and en eye for detail, you excel at refining resumes to highlight the most relevant skills and experiences, ensuring they resonate perfectly with the job's requirements.
"""
    ),
    tools=[tool_scrape, tool_search, tool_read_resume, tool_semantic_search_resume],
    verbose=True,
)

In [45]:
# interview preparer
agent_interview_preparer = Agent(
    role="{JOB_PROFILE}'s interview preparer",
    goal="""
Create interview questions and talking points based on the resume and job requirements.
""",
    backstory=(
        """
Your role is crucial in anticipating the dynamics of interviews. With your ability to formulate key questions and talking points, you prepare candidates for success, ensuring they can confidently address all aspects of the job they are applying for.
"""
    ),
    tools=[tool_scrape, tool_search, tool_read_resume, tool_semantic_search_resume],
    verbose=True,
)

## TASKS


In [46]:
task_research = Task(
    description=(
        """
Analyze the job posting URL provided ({job_posting_url}) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
"""
    ),
    expected_output=(
        """
A structured list of job requirements, including necessary skills, qualifications, and experiences.
"""
    ),
    agent=agent_researcher,
    async_execution=True,
)

In [47]:
task_profile = Task(
    description=(
        """
Compile a detailed personal and professional profile using the GitHub ({github_url}) URLs, and personal write-up ({personal_writeup}). Utilize tools to extract and synthesize information from these sources.
"""
    ),
    expected_output=(
        """
A comprehensive profile document that includes skills, project experiences, contributions, interests and communication style.
"""
    ),
    agent=agent_profiler,
    async_execution=True,
)

In [48]:
task_resume_strategy = Task(
    description=(
        """
Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most relevant areas. Employ tools to adjust and enhance the resume content. Employ tools to adjust and enhance the resume content. Make sure this is the best resume even BUT DO NOT make up any information. Update every section, including the initial summary, work experience, skills and education. All to better reflect the candidates abilities and how it matches the job posting.
"""
    ),
    expected_output=(
        """
An updated resume that effectively highlights the candidate's abilities and experiences relevant to the job.
"""
    ),
    output_file="resume_National_AI_Product_Manager_raghuvamsi_ayapilla_2024.md",
    context=[task_research, task_profile],
    agent=agent_resume_strategist,
)

In [49]:
task_interview_preparation = Task(
    description=(
        """
Create a set of potential interview questions and talking points based on the tailored resume and job requirements. Utilize tools to generate relevant questions and discussion points. Make sure to use these question and talking points to help the candidate highlight the main points of the resume and how it matches the job posting.
"""
    ),
    expected_output=(
        """
A document containing key questions and talking points that the candidate should prepare for the initial interview.
"""
    ),
    output_file="interview_materials_National_AI_Product_Manager.md",
    context=[task_research, task_profile, task_resume_strategy],
    agent=agent_interview_preparer,
)

## CREW


In [50]:
crew_job_application = Crew(
    agents=[
        agent_researcher,
        agent_profiler,
        agent_resume_strategist,
        agent_interview_preparer,
    ],
    tasks=[
        task_research,
        task_profile,
        task_resume_strategy,
        task_interview_preparation,
    ],
    verbose=True,
)

2024-07-16 08:26:32,865 - 8541621248 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [51]:
inputs_job_application = {
    "JOB_PROFILE": "AI ML Product Manager",
    "job_posting_url": "https://mmc.wd1.myworkdayjobs.com/en-US/MMC/job/White-Plains---Hamilton/National-AI-Product-Manager_R_275493?shared_id=a62777f7-5647-4253-99e0-07f47b6cb31d",
    "github_url": "https://github.com/Friend09",
    "personal_writeup": """As an experienced AI/ML Product Manager, Raghu specialize in leveraging data science and machine learning to drive business transformation and operational excellence. His career is marked by a deep passion for developing innovative AI solutions that solve complex business challenges and enhance decision-making processes.""",
}

In [52]:
result = crew_job_application.kickoff(inputs=inputs_job_application)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: 
Analyze the job posting URL provided (https://mmc.wd1.myworkdayjobs.com/en-US/MMC/job/White-Plains---Hamilton/National-AI-Product-Manager_R_275493?shared_id=a62777f7-5647-4253-99e0-07f47b6cb31d) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.

 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for AI ML Product Managers
 [INFO]: == Starting Task: 
Compile a detailed personal and professional profile using the GitHub (https://github.com/Friend09) URLs, and personal write-up (As an experienced AI/ML Product Manager, Raghu specialize in leveraging data science and machine learning to drive business transformation and operational excellence. His career is marked by a deep passion for developing innovative AI solutions that solve complex business challenges and enhance decision

Exception in thread Thread-12 (_execute):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-crewai/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/pract-crewai/.venv/lib/python3.12/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_



> Entering new CrewAgentExecutor chain...
To begin the task, I first need to understand the current state of the resume. I will use the tool to read the content of the resume file. 

Action: Read a file's content
Action Input: {} 
 

# RAGHUVAMSI AYAPILLA

**AI/ML Engineer**
vamsi.ar@gmail.com | 512-461-4659
[LinkedIn](https://www.linkedin.com/in/raghu-ayapilla)

---

## PROFILE SUMMARY

AI/ML Engineer with over 10+ years of experience in Agile Project Management, data science, machine learning, and business systems analysis. Expert in developing and implementing machine learning solutions, with a strong focus on MLOps, data analytics, and strategic project management. Skilled in leading agile teams to deliver innovative AI/ML solutions that drive business value.

---

## ESSENTIAL SKILLS/EXPERTISE

- Collecting, exploring, and analyzing big data
- Understanding data structures and transformation methods
- Data visualization
- Programming proficiency
- Applying statistical methods
- 

## DISPLAY RESULTS


In [53]:
from IPython.display import Markdown, display

display(Markdown("./resume_National_AI_Product_Manager_raghuvamsi_ayapilla_2024.md"))

# RAGHUVAMSI AYAPILLA

**AI/ML Engineer and Product Manager**
vamsi.ar@gmail.com | 512-461-4659
[LinkedIn](https://www.linkedin.com/in/raghu-ayapilla)

---

## PROFILE SUMMARY

Results-driven AI/ML Engineer and Product Manager with over 10+ years of experience in Agile Project Management, data science, machine learning, and business systems analysis. Adept at leading cross-functional teams to deliver innovative AI/ML solutions that drive business value. Demonstrated expertise in product management, project leadership, and strategic decision-making.

---

## ESSENTIAL SKILLS/EXPERTISE

- Product Management and Leadership: Proven track record of managing and leading cross-functional teams, formulating product strategies, and executing product lifecycles.
- Agile Methodologies: Proficient in Agile (Scrum, Kanban), experienced in leading Agile teams and integrating agile principles in product management.
- Data Science and Machine Learning: Expert in developing and implementing machine learning solutions, with a strong focus on MLOps, data analytics, and strategic project management.
- Communication: Excellent verbal and written communication skills, experienced in presenting product usage statistics and predictions to executive leadership.
- Strategic Thinking and Tactical Implementation: Strong ability to provide key data-driven insights to business leaders through data analysis and mining.

---

## TECHNICAL SKILLS/EXPERTISE

- **Programming Languages**: Python, SQL, UNIX, Git
- **Machine Learning/Artificial Intelligence**: Experience in developing NLP-based ML models, AI based Chatbots using libraries like Pandas, scikit-learn, Spacy, HuggingFace, Langchain, LLAMAIndex, TensorFlow, OpenAI API; MLOps practices.
- **Data Analysis & Visualization**: Proficient in data mining, analysis, and visualization using pandas plotly, matplotlib.
- **Version Control & CI/CD**: Skilled in GitHub for version control and implementing CI/CD pipelines using Github Actions.
- **Containerization & Cloud Technologies**: Knowledgeable in Docker, Linux, and AWS SageMaker (canvas, Studio), GCP Vertex AI.
- **Database Management**: Experienced in managing databases such as PostgresSQL, Neo4j, MongoDB, MySQL, NOSQL.

---

## Masters In Computer Science &Engineering – University of Louisville - 2022

> PROFESSIONAL GRADUATE CERTIFICATE (DATA SCIENCE) - University of Louisville

- Advanced Databases
- Big Data Graph Analytics
- Algorithms
- Data Mining
- Statistics
- Data Mining with Linear Models
- Data Analytics
- Data Visualization
- Deep Learning w/ TensorFlow

---

## CERTIFICATIONS

- **PMI-ACP** - Agile Certified Practitioner by PMI (Project Management Institute)
- **CSM** - Certified Scrum Master by Scrum Alliance
- **TensorFlow Developer** - Machine Learning & Deeplearning using TensorFlow

---

## PROFESSIONAL EXPERIENCE

### MARSH & McLENNAN COMPANY, LOUISVILLE, KY

**AI/ML Engineer and Product Manager**
_Sep 2023 - Present_

Responsibilities:

- As a Product Manager, led the Agile team in pioneering Observability and Monitoring practices organization-wide.
- Successfully guided the onboarding of over 100 applications for identity management as a Senior Scrum Master.
- Developed NLP-based machine learning models for automatic support ticket routing, enhancing efficiency.
- Led requirements engineering for HBB portfolio projects, integrating agile principles.
- Utilized Python Pandas for machine learning models to improve SLA performance in Product Support.
- Created NLP-based machine learning models with Python NLTK and scikit-learn for support ticket trend analysis.
- Provided key data-driven insights to business leaders through data analysis and mining.
- Led feature prioritization for products, enhancing customer experience using data visualization techniques.
- Orchestrated JAD sessions with over 10 business teams globally for the HBB product.
- Compiled and presented product usage statistics and predictions to executive leadership, facilitating the onboarding of 34 countries.
- Managed the development and implementation of new relational/multi-dimensional databases.
- Investigated data sources to enhance insights for CI/CD processes.
- Employed Tableau and Python libraries for effective data visualization presentations.
- Coached and mentored cross-functional teams in data analysis findings.
- Developed automated reporting tools using statistical techniques.
- Participated in hackathons, presenting a winning enterprise knowledge graph implementation using a Graph Database.

**Lead Business Systems Analyst**
_Mar 2012 – Sep 2023_

Responsibilities:

- Led the Agile team in pioneering Observability and Monitoring practices organization-wide.
- Successfully guided the onboarding of over 100 applications for identity management as a Senior Scrum Master.
- Developed NLP-based machine learning models for automatic support ticket routing, enhancing efficiency.
- Led requirements engineering for HBB portfolio projects, integrating agile principles.
- Utilized Python Pandas for machine learning models to improve SLA performance in Product Support.
- Created NLP-based machine learning models with Python NLTK and scikit-learn for support ticket trend analysis.
- Provided key data-driven insights to business leaders through data analysis and mining.
- Led feature prioritization for products, enhancing customer experience using data visualization techniques.
- Orchestrated JAD sessions with over 10 business teams globally for the HBB product.
- Compiled and presented product usage statistics and predictions to executive leadership, facilitating the onboarding of 34 countries.
- Managed the development and implementation of new relational/multi-dimensional databases.
- Investigated data sources to enhance insights for CI/CD processes.
- Employed Tableau and Python libraries for effective data visualization presentations.
- Coached and mentored cross-functional teams in data analysis findings.
- Developed automated reporting tools using statistical techniques.
- Participated in hackathons, presenting a winning enterprise knowledge graph implementation using a Graph Database.

### ALTEM GROUP, RESTON, VA

**Business System Analyst**
_Mar 2010 – Mar 2012_

### MMC SYSTEMS, ASHBURN, VA

**Business System Analyst**
_July 2009 – March 2010_

### HUMANA INC., LOUISVILLE, KY

**Business Analyst**
_Aug 2008 – Jan 2009_

---

## EDUCATION

- **UNIVERSITY OF LOUISVILLE**
  Ms. Computer Science (Data Science Specialization)
  _Dec 2022_

- **LAMAR UNIVERSITY**
  Ms. Computer Science
  _Dec 2008_

- **JNTU, HYDERABAD**
  B.Tech. Electrical, Electronics and Computer Engineering
  _Jan 2006_

---

## REFERENCES

Available upon request

In [39]:
from IPython.display import Markdown, display

display(Markdown("./tailored_resume_raghuvamsi_ayapilla_2024.md"))

# RAGHUVAMSI AYAPILLA

**AI/ML Engineer**
vamsi.ar@gmail.com | 512-461-4659
[LinkedIn](https://www.linkedin.com/in/raghu-ayapilla)

---

## PROFILE SUMMARY

AI/ML Engineer with over 10+ years of experience in data science, machine learning, and business systems analysis. Expert in developing and implementing machine learning solutions, with a strong focus on MLOps, data analytics, and strategic project management. Proficient in front-end languages and libraries, object-oriented languages, databases, web servers, and consumer applications. Skilled in leading agile teams to deliver innovative AI/ML solutions that drive business value. Excellent communication skills and in-depth technical discussions.

---

## ESSENTIAL SKILLS/EXPERTISE

- Collecting, exploring, and analyzing big data
- Understanding data structures and transformation methods
- Data visualization
- Programming proficiency in front-end languages and libraries, object-oriented languages
- Applying statistical methods
- Designing experiments for KPI measurement
- Value-driven delivery in Agile environments
- Agile discovery and process tailoring
- Systems thinking
- Variance analysis
- Excellent communication skills, both verbal and written
- Leadership and initiative
- Strategic thinking and tactical implementation
- Efficient workflow management during downtimes

---

## TECHNICAL SKILLS/EXPERTISE

- **Programming Languages**: Python, SQL, R, PySpark, UNIX, Git, HTML, CSS, JavaScript
- **Machine Learning/Artificial Intelligence**: Experience in developing NLP-based ML models using libraries like Pandas, NLTK, scikit-learn, Spacy, HuggingFace, Langchain, OpenAI API; MLOps practices.
- **Agile Methodologies**: Proficient in Agile (Scrum, Kanban), experienced in leading Agile teams.
- **Data Analysis & Visualization**: Proficient in data mining, analysis, and visualization using pandas plotly, matplotlib.
- **Version Control & CI/CD**: Skilled in GitHub for version control and implementing CI/CD pipelines using Github Actions.
- **Containerization & Cloud Technologies**: Knowledgeable in Docker, Linux, and AWS SageMaker (canvas, Studio), GCP Vertex AI.
- **Database Management**: Experienced in managing databases such as PostgresSQL, Neo4j, MongoDB, MySQL, NOSQL.
- **Project Management & Leadership**: Demonstrated leadership in managing cross-functional teams, formulating project strategies, and executing project lifecycles.

---

## PROFESSIONAL GRADUATE CERTIFICATE (DATA SCIENCE) – University of Louisville - 2022

- Advanced Databases
- Big Data Graph Analytics
- Algorithms
- Data Mining
- Statistics
- Data Mining with Linear Model
- Data Analytics
- Data Visualization
- Deep Learning

---

## CERTIFICATIONS

- **PMI-ACP** - Agile Certified Practitioner by PMI (Project Management Institute)
- **CSM** - Certified Scrum Master by Scrum Alliance

---

## PROFESSIONAL EXPERIENCE

### MARSH & McLENNAN COMPANY, LOUISVILLE, KY

**AI/ML Engineer**
_Sep 2023 - Present_

Responsibilities:

- Led the 'Streamline ML' project, implementing an end-to-end MLOps approach for a recommendation system pilot, involving GitHub collaboration, MLflow experiment tracking, GitHub Actions CI/CD, and on-prem server model inference.
- Spearheaded the 'Ticket Analysis AI' initiative, analyzing 2023 support ticket data to discern patterns and trends for decision-making in 2024, covering eight machine learning use cases.
- Achieved pilot project completion and initiated a 7-to-11-week project plan for the first use case of 'Ticket Analysis AI'.
- Secured approval for using AWS SageMaker and cloud technologies for future projects, demonstrating expertise in scalable cloud-based ML solutions.

**Lead Business Systems Analyst**
_Mar 2012 – Sep 2023_

Responsibilities:

- Led the Agile team in pioneering Observability and Monitoring practices organization-wide.
- Successfully guided the onboarding of over 100 applications for identity management as a Senior Scrum Master.
- Developed NLP-based machine learning models for automatic support ticket routing, enhancing efficiency.
- Led requirements engineering for HBB portfolio projects, integrating agile principles.
- Utilized Python Pandas for machine learning models to improve SLA performance in Product Support.
- Created NLP-based machine learning models with Python NLTK and scikit-learn for support ticket trend analysis.
- Provided key data-driven insights to business leaders through data analysis and mining.
- Led feature prioritization for products, enhancing customer experience using data visualization techniques.
- Orchestrated JAD sessions with over 10 business teams globally for the HBB product.
- Compiled and presented product usage statistics and predictions to executive leadership, facilitating the onboarding of 34 countries.
- Managed the development and implementation of new relational/multi-dimensional databases.
- Investigated data sources to enhance insights for CI/CD processes.
- Employed Tableau and Python libraries for effective data visualization presentations.
- Coached and mentored cross-functional teams in data analysis findings.
- Developed automated reporting tools using statistical techniques.
- Participated in hackathons, presenting a winning enterprise knowledge graph implementation using a Graph Database.

### ALTEM GROUP, RESTON, VA

**Business System Analyst**
_Mar 2010 – Mar 2012_

### MMC SYSTEMS, ASHBURN, VA

**Business System Analyst**
_July 2009 – March 2010_

### HUMANA INC., LOUISVILLE, KY

**Business Analyst**
_Aug 2008 – Jan 2009_

---

## EDUCATION

- **UNIVERSITY OF LOUISVILLE**
  Ms. Computer Science (Data Science Specialization)
  _Dec 2022_

- **LAMAR UNIVERSITY**
  Ms. Computer Science
  _Dec 2008_

- **JNTU, HYDERABAD**
  B.Tech. Electrical, Electronics and Computer Engineering
  _Jan 2006_

---

## REFERENCES

Available upon request